In [9]:
# Import Dependencies
import pandas as pd
import numpy as np

In [11]:
# Read the csv into a DataFrame
movies_df = pd.read_csv('Merged_Movies_Raw_Dataset.csv')
movies_df

,Unnamed: 0,budget_ds-movies,company,country,director,genre,gross,rating,score,star,...,original_language,overview,popularity,revenue,status,tagline,title,vote_average,vote_count,Ave_runtime
0,0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,R,8.1,Wil Wheaton,...,en,"After the death of a friend, a writer recounts...",51.109279,52287414,Released,"For some, it's the last real taste of innocenc...",Stand by Me,7.8,1522,89.0
1,1,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,PG,6.9,Tom Cruise,...,en,For Lieutenant Pete 'Maverick' Mitchell and hi...,58.900647,356830601,Released,Up there with the best of the best.,Top Gun,6.7,1698,110.0
2,2,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,R,8.4,Sigourney Weaver,...,en,When Ripley's lifepod is found by a salvage cr...,67.660940,183316455,Released,This Time It's War,Aliens,7.7,3220,137.0
3,3,6000000.0,Hemdale,UK,Oliver Stone,Drama,138530565.0,R,8.1,Charlie Sheen,...,en,"As a young and naive recruit in Vietnam, Chris...",49.802914,138530565,Released,The first casualty of war is innocence.,Platoon,7.5,1205,120.0
4,4,8800000.0,Rimfire Films,Australia,Peter Faiman,Adventure,174635000.0,PG-13,6.5,Paul Hogan,...,en,When a New York reporter plucks crocodile hunt...,16.072466,328203506,Released,There's a little of him in all of us.,Crocodile Dundee,6.3,503,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3183,3183,8000000.0,Cinelou Films,USA,Bruce Beresford,Comedy,685143.0,PG-13,7.7,Eddie Murphy,...,en,A unique friendship develops when a little gir...,7.828459,0,Released,He was the one person she could always count on.,Mr. Church,7.0,129,104.0
3184,3184,18000000.0,Gold Circle Films,USA,Kirk Jones,Comedy,59573085.0,PG-13,6.0,Nia Vardalos,...,en,The continuing adventures of the Portokalos fa...,16.027955,88923251,Released,People change. Greeks don't.,My Big Fat Greek Wedding 2,5.5,322,94.0
3185,3185,0.0,Bing Feng Bao Entertainment,USA,James Schamus,Drama,3399841.0,R,6.8,Logan Lerman,...,en,"In 1951, Marcus Messner, a working-class Jewis...",5.625989,0,Released,Based on the novel by Philip Roth.,Indignation,6.8,69,110.0
3186,3186,0.0,StudioCanal,UK,Susanna White,Crime,3152725.0,R,6.2,Carlos Acosta,...,en,A young Oxford academic and his attorney girlf...,10.547959,9930095,Released,Who Can You Trust With The Truth?,Our Kind of Traitor,6.0,160,108.0


## Prepare data for machine learning

In [12]:
# Review columns and decide which columns to use for the model
movies_df.columns.tolist()

['Unnamed: 0',
 'budget_ds-movies',
 'company',
 'country',
 'director',
 'genre',
 'gross',
 'rating',
 'score',
 'star',
 'votes',
 'writer',
 'year',
 'id',
 'keywords',
 'original_language',
 'overview',
 'popularity',
 'revenue',
 'status',
 'tagline',
 'title',
 'vote_average',
 'vote_count',
 'Ave_runtime']

In [13]:
# Check data type for columns
movies_df.dtypes

Unnamed: 0             int64
budget_ds-movies     float64
company               object
country               object
director              object
genre                 object
gross                float64
rating                object
score                float64
star                  object
votes                  int64
writer                object
year                   int64
id                     int64
keywords              object
original_language     object
overview              object
popularity           float64
revenue                int64
status                object
tagline               object
title                 object
vote_average         float64
vote_count             int64
Ave_runtime          float64
dtype: object

In [16]:
# Apply bucketing to company, country, director, genre, rating, star, writer, original_language
# Company
company_counts = movies_df.company.value_counts()
company_counts.head(60)

Universal Pictures                        197
Warner Bros.                              194
Paramount Pictures                        166
Twentieth Century Fox Film Corporation    132
Columbia Pictures Corporation             113
New Line Cinema                           111
Columbia Pictures                          96
Walt Disney Pictures                       74
Touchstone Pictures                        69
DreamWorks                                 61
Metro-Goldwyn-Mayer (MGM)                  54
Miramax                                    48
Fox 2000 Pictures                          48
Fox Searchlight Pictures                   39
Screen Gems                                38
Dimension Films                            37
Lionsgate                                  34
Summit Entertainment                       31
Castle Rock Entertainment                  31
Revolution Studios                         26
Focus Features                             25
TriStar Pictures                  

In [19]:
# Country
country_counts = movies_df.country.value_counts()
country_counts.head(30)

USA               2609
UK                 298
France              62
Germany             44
Canada              39
Japan               13
Spain               13
Australia           12
Hong Kong           10
China                9
India                8
Ireland              7
Italy                7
New Zealand          5
Mexico               5
Denmark              5
Russia               4
South Korea          4
Sweden               3
Norway               3
Brazil               3
Netherlands          2
Iran                 2
Taiwan               2
Austria              2
Argentina            2
Finland              2
Czech Republic       2
Colombia             1
Greece               1
Name: country, dtype: int64

In [18]:
# Director
director_counts = movies_df.director.value_counts()
director_counts.head(60)

Steven Spielberg      19
Woody Allen           18
Clint Eastwood        17
Steven Soderbergh     15
Tim Burton            15
Renny Harlin          13
Joel Schumacher       13
Oliver Stone          13
Ridley Scott          13
Spike Lee             13
Ron Howard            12
Robert Zemeckis       12
Martin Scorsese       12
Rob Reiner            11
Richard Linklater     11
Kevin Smith           11
Shawn Levy            11
Michael Bay           11
Chris Columbus        10
Barry Levinson        10
Bobby Farrelly        10
Tony Scott            10
Stephen Frears         9
M. Night Shyamalan     9
Robert Rodriguez       9
Paul W.S. Anderson     9
Sam Raimi              9
Rob Cohen              9
Dennis Dugan           9
David Fincher          9
Richard Donner         8
John McTiernan         8
Gus Van Sant           8
Antoine Fuqua          8
Ivan Reitman           8
Wes Craven             8
Christopher Nolan      8
Stephen Herek          8
Donald Petrie          8
Jay Roach              8


In [21]:
# Genre
genre_counts = movies_df.genre.value_counts()
genre_counts

Comedy       882
Action       790
Drama        552
Crime        228
Adventure    205
Biography    178
Animation    168
Horror       142
Mystery       15
Fantasy       12
Sci-Fi         6
Family         4
Romance        3
Thriller       2
Western        1
Name: genre, dtype: int64